# Levantamiento Acciones Comerciales - Código

> **PROYECTO : LEVANTAMIENTO CÓDIGO EN EL GESTOR DE CAMPANAS TERADATA** <br> 
**Extracción de tablas, campos y reglas utilizadas en las Vistas, Bteq, SP e Input Analítico** <br>
Los archivos a procesar están en la carpeta ./ArchivosProcesar  <br>
Versión:  1.0  <br>
Fecha: 01-05-2020  <br>
Descripción: Versión Inicial  <br>
Desarrollador: Axity | Adriana Jiménez 

## Librerías

In [10482]:
import re
import sys
import os, glob
import pandas as pd 
import numpy as np
from decimal import Decimal as D
import datetime

## Librerias de parseo de queries
import sqlparse
from sqlparse.sql import TokenList
from sqlparse.tokens import Name, Whitespace, Wildcard, Number, Punctuation, Text, Operator, Literal
from sqlparse.tokens import DML, DDL, Keyword
import sql_metadata as sqllib


## Libreria que se integra con Teradata
import giraffez

import warnings
warnings.filterwarnings('ignore')

## Variables Globales

In [10483]:
noConditionInput = 'Parametros: No tiene condicion de tabla input'

In [10484]:
def unique(_list):
    """
    Hace que una lista tenga registro unicos y mantengan el orden
    """
    ret = []

    for item in _list:
        if item not in ret:
            ret.append(item)

    return ret

## Funciones: Insertar en BD Teradata

In [10485]:
## Configuracion de conexion
td_config = {
    "username": "exajibl",
    "password": "acjb0610",
    "host": "dataware.bci.cl"}

In [10486]:
def create_and_drop_tables_teradata():
    '''
    Función que crea las tablas en Teradata
    '''
    
    print("ELIMINANDO Y CREANDO TABLAS..")
    drop_sql_bteq    = "DROP TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_BTEQ"    
    drop_sql_sp      = "DROP TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_SP"
    drop_sql_vistas  = "DROP TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_VISTAS"
    
        
    create_sql_bteq = """CREATE MULTISET TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_BTEQ
        (
          TIPO_ARCHIVO VARCHAR(20) ,
          ARCHIVO VARCHAR(200),
          LINEAS_DE_CODIGO VARCHAR(100),
          NUMERO_PASO VARCHAR(10),      
          SENTENCIA_DML VARCHAR(15) ,
          ESQUEMA_OUTPUT VARCHAR(100) ,
          TABLA_OUTPUT VARCHAR(100) ,
          MULTINIVEL VARCHAR(30),
          ESQUEMA_INPUT VARCHAR(100) ,
          TABLA_INPUT VARCHAR(100) ,
          ALIAS_SUBQUERY VARCHAR(100) ,
          COLUMNA VARCHAR(100) ,
          VARIABLE VARCHAR(100) ,
          LOGICA_NEGOCIO VARCHAR(28000),
          ALIAS_CAMPO VARCHAR(100)   
          ) ;"""       
   
    create_sql_sp = """CREATE MULTISET TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_SP
        (
          TIPO_ARCHIVO VARCHAR(20) ,
          ARCHIVO VARCHAR(200),
          LINEAS_DE_CODIGO VARCHAR(100),          
          NUMERO_PASO VARCHAR(10),      
          SENTENCIA_DML VARCHAR(15) ,
          ESQUEMA_OUTPUT VARCHAR(100) ,
          TABLA_OUTPUT VARCHAR(100) ,
          MULTINIVEL VARCHAR(30),
          ESQUEMA_INPUT VARCHAR(100) ,
          TABLA_INPUT VARCHAR(100) ,
          ALIAS_SUBQUERY VARCHAR(100) ,
          COLUMNA VARCHAR(100) ,
          VARIABLE VARCHAR(100) ,
          LOGICA_NEGOCIO VARCHAR(28000),
          ALIAS_CAMPO VARCHAR(100)      
          ) ;"""
    
    create_sql_vistas = """CREATE MULTISET TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_VISTAS
        (
          TIPO_ARCHIVO VARCHAR(20) ,
          ARCHIVO VARCHAR(200),   
          LINEAS_DE_CODIGO VARCHAR(100),  
          SENTENCIA_DML VARCHAR(15) ,
          ESQUEMA_OUTPUT VARCHAR(30) ,
          TABLA_OUTPUT VARCHAR(50) ,
          MULTINIVEL VARCHAR(30),
          ESQUEMA_INPUT VARCHAR(50) ,
          TABLA_INPUT VARCHAR(50),
          ALIAS_SUBQUERY VARCHAR(50) ,
          COLUMNA VARCHAR(50) ,
          VARIABLE VARCHAR(50) ,
          LOGICA_NEGOCIO VARCHAR(28000),
          ALIAS_CAMPO VARCHAR(50)        
          ) ;"""
    
    with giraffez.Cmd(**td_config) as cmd:
        if cmd.exists("EDW_TEMPUSU.LAC_LEVANTAMIENTO_BTEQ"):
            cmd.execute(drop_sql_bteq)
        if cmd.exists("EDW_TEMPUSU.LAC_LEVANTAMIENTO_SP"):
            cmd.execute(drop_sql_sp)
        if cmd.exists("EDW_TEMPUSU.LAC_LEVANTAMIENTO_VISTAS"):
            cmd.execute(drop_sql_vistas)
    

        cmd.execute(create_sql_bteq)
        cmd.execute(create_sql_sp)
        cmd.execute(create_sql_vistas)

In [10487]:
def insert_files_csv_in_teradata():
    '''
    Función que inserta los dataframes a Teradata
    '''
    
    print("INSERTANDO ARCHIVOS A LAS TABLAS..\n")
    
    df_bteq    = pd.read_csv("LAC_Levantamiento_Bteq.csv",  sep='|' ) 
    df_vistas  = pd.read_csv("LAC_Levantamiento_Vistas.csv",  sep='|') 
    df_sp      = pd.read_csv("LAC_Levantamiento_StoredProcedures.csv" ,  sep='|') 

    
    if (df_bteq.empty == False):         
        
        print("INSERTANDO EN EDW_TEMPUSU.LAC_LEVANTAMIENTO_BTEQ")
        
        df_bteq['TIPO_ARCHIVO']    = df_bteq['TIPO_ARCHIVO'].astype('str')
        df_bteq['ARCHIVO']         = df_bteq['ARCHIVO'].astype('str')            
        df_bteq['LINEAS_DE_CODIGO'] = df_bteq['LINEAS_DE_CODIGO'].astype('str')    
        df_bteq['NUMERO_PASO']     = df_bteq['NUMERO_PASO'].astype('str')
        df_bteq['SENTENCIA_DML']   = df_bteq['SENTENCIA_DML'].astype('str')
        df_bteq['ESQUEMA_OUTPUT']  = df_bteq['ESQUEMA_OUTPUT'].astype('str')
        df_bteq['TABLA_OUTPUT']    = df_bteq['TABLA_OUTPUT'].astype('str')
        df_bteq['MULTINIVEL']      = df_bteq['MULTINIVEL'].astype('str')
        df_bteq['ESQUEMA_INPUT']   = df_bteq['ESQUEMA_INPUT'].astype('str')
        df_bteq['TABLA_INPUT']     = df_bteq['TABLA_INPUT'].astype('str')
        df_bteq['ALIAS_SUBQUERY']  = df_bteq['ALIAS_SUBQUERY'].astype('str')
        df_bteq['COLUMNA']         = df_bteq['COLUMNA'].astype('str')
        df_bteq['VARIABLE']        = df_bteq['VARIABLE'].astype('str')
        df_bteq['LOGICA_NEGOCIO']  = df_bteq['LOGICA_NEGOCIO'].astype('str')
        df_bteq['ALIAS_CAMPO']     = df_bteq['ALIAS_CAMPO'].astype('str')
        
        
        with giraffez.BulkLoad("EDW_TEMPUSU.LAC_LEVANTAMIENTO_BTEQ", **td_config) as load:
            load.cleanup()
            load.columns = df_bteq.columns.tolist()
            for row in df_bteq.values.tolist(): 
                load.put([row[0], row[1], row[2], row[3], row[4], row[5], row[6],
                          row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14] ])
        

    if (df_sp.empty == False):
        
        print("INSERTANDO EN EDW_TEMPUSU.LAC_LEVANTAMIENTO_SP")
        
        df_sp['TIPO_ARCHIVO']    = df_sp['TIPO_ARCHIVO'].astype('str')
        df_sp['ARCHIVO']         = df_sp['ARCHIVO'].astype('str')   
        df_sp['LINEAS_DE_CODIGO']= df_sp['LINEAS_DE_CODIGO'].astype('str')
        df_sp['NUMERO_PASO']     = df_sp['NUMERO_PASO'].astype('str')
        df_sp['SENTENCIA_DML']   = df_sp['SENTENCIA_DML'].astype('str')
        df_sp['ESQUEMA_OUTPUT']  = df_sp['ESQUEMA_OUTPUT'].astype('str')
        df_sp['TABLA_OUTPUT']    = df_sp['TABLA_OUTPUT'].astype('str')
        df_sp['MULTINIVEL']      = df_sp['MULTINIVEL'].astype('str')
        df_sp['ESQUEMA_INPUT']   = df_sp['ESQUEMA_INPUT'].astype('str')
        df_sp['TABLA_INPUT']     = df_sp['TABLA_INPUT'].astype('str')
        df_sp['ALIAS_SUBQUERY']  = df_sp['ALIAS_SUBQUERY'].astype('str')
        df_sp['COLUMNA']         = df_sp['COLUMNA'].astype('str')
        df_sp['VARIABLE']        = df_sp['VARIABLE'].astype('str')
        df_sp['LOGICA_NEGOCIO']  = df_sp['LOGICA_NEGOCIO'].astype('str')
        df_sp['ALIAS_CAMPO']     = df_sp['ALIAS_CAMPO'].astype('str')
        
        
        with giraffez.BulkLoad("EDW_TEMPUSU.LAC_LEVANTAMIENTO_SP", **td_config) as load:
            load.cleanup()
            load.columns = df_sp.columns.tolist()
            for row in df_sp.values.tolist(): 
                load.put([row[0], row[1], row[2], row[3], row[4], row[5], row[6],
                          row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14] ])  
        
        
    
    if (df_vistas.empty == False): 
        
        print("INSERTANDO EN EDW_TEMPUSU.LAC_LEVANTAMIENTO_VISTAS")
        
        df_vistas['TIPO_ARCHIVO']    = df_vistas['TIPO_ARCHIVO'].astype('str')
        df_vistas['ARCHIVO']         = df_vistas['ARCHIVO'].astype('str')
        df_vistas['LINEAS_DE_CODIGO']= df_vistas['LINEAS_DE_CODIGO'].astype('str')
        df_vistas['SENTENCIA_DML']   = df_vistas['SENTENCIA_DML'].astype('str')
        df_vistas['ESQUEMA_OUTPUT']  = df_vistas['ESQUEMA_OUTPUT'].astype('str')
        df_vistas['TABLA_OUTPUT']    = df_vistas['TABLA_OUTPUT'].astype('str')
        df_vistas['MULTINIVEL']      = df_vistas['MULTINIVEL'].astype('str')
        df_vistas['ESQUEMA_INPUT']   = df_vistas['ESQUEMA_INPUT'].astype('str')
        df_vistas['TABLA_INPUT']     = df_vistas['TABLA_INPUT'].astype('str')
        df_vistas['ALIAS_SUBQUERY']  = df_vistas['ALIAS_SUBQUERY'].astype('str')
        df_vistas['COLUMNA']         = df_vistas['COLUMNA'].astype('str')
        df_vistas['VARIABLE']        = df_vistas['VARIABLE'].astype('str')
        df_vistas['LOGICA_NEGOCIO']  = df_vistas['LOGICA_NEGOCIO'].astype('str')
        df_vistas['ALIAS_CAMPO']     = df_vistas['ALIAS_CAMPO'].astype('str')
        
        
        with giraffez.BulkLoad("EDW_TEMPUSU.LAC_LEVANTAMIENTO_VISTAS", **td_config) as load:
            load.cleanup()
            load.columns = df_vistas.columns.tolist()
            for row in df_vistas.values.tolist(): 
                load.put([row[0], row[1], row[2], row[3], row[4], row[5], row[6],
                          row[7], row[8], row[9], row[10],row[11], row[12], row[13]  ])
           
              
 


## Funciones de procesamiento de query

In [10488]:
def remove_comments(text):
    """ remove c-style comments.
        text: blob of text with comments (can include newlines)
        returns: text with comments removed
    """
    pattern = r"""
                            ##  --------- COMMENT ---------
           /\*              ##  Start of /* ... */ comment
           [^*]*\*+         ##  Non-* followed by 1-or-more *'s
           (                ##
             [^/*][^*]*\*+  ##
           )*               ##  0-or-more things which don't start with /
                            ##    but do end with '*'
           /                ##  End of /* ... */ comment
         |                  ##  -OR-  various things which aren't comments:
           (                ## 
                            ##  ------ " ... " STRING ------
             "              ##  Start of " ... " string
             (              ##
               \\.          ##  Escaped char
             |              ##  -OR-
               [^"\\]       ##  Non "\ characters
             )*             ##
             "              ##  End of " ... " string
           |                ##  -OR-
                            ##
                            ##  ------ ' ... ' STRING ------
             '              ##  Start of ' ... ' string
             (              ##
               \\.          ##  Escaped char
             |              ##  -OR-
               [^'\\]       ##  Non '\ characters
             )*             ##
             '              ##  End of ' ... ' string
           |                ##  -OR-
                            ##
                            ##  ------ ANYTHING ELSE -------
             .              ##  Anything other char
             [^/"'\\]*      ##  Chars which doesn't start a comment, string
           )                ##    or escape
    """
    regex = re.compile(pattern, re.VERBOSE|re.MULTILINE|re.DOTALL)
    noncomments = [m.group(2) for m in regex.finditer(text) if m.group(2)]

    return "".join(noncomments)

In [10489]:
def deleteInconsistencyQuery(queryString):
    '''
    Función que elimina caracteres varios de una query
    '''    
    characters = '[%#${}]'
    finalQuery = re.sub(characters, '', str(queryString))
    return finalQuery

In [10490]:
def preprocess_query(queryString):
    '''
    Función que limpia y ordena una query para ser procesada posteriormente
    '''
    queryString = queryString.replace('"','')
    queryString = sqllib.preprocess_query(queryString)
    queryString = deleteInconsistencyQuery(queryString)
    return queryString

In [10491]:
def mergeDataframeTablesColumnsQuery(df_final_tables, df_final_columns):
    '''
    Función que une las tablas con los campos según el multinivel al que corresponde
    '''
    
    columnsDF = ['MULTINIVEL','ESQUEMA_INPUT','TABLA_INPUT','ALIAS_SUBQUERY',
                 'NOMBRE_CAMPO', 'CAMPO_COMPUESTO','LOGICA_NEGOCIO','ALIAS_CAMPO']
    
    
    # Divide la columna "CAMPOS" haciendo split '.'
    # Obtengo un ALIAS temporal del CAMPO
    
    df_tmp_columns = df_final_columns.CAMPOS.str.split(".", n=3, expand=True)

    
    if (df_tmp_columns.empty == False):
        # Para los casos que viene el campo en formato: esquema.tabla.campo
        if (len(df_tmp_columns.columns) == 3): 
            df_final_columns['ALIAS_TMP']  = 'N/A'
            df_final_columns[['COLUMNA_1','COLUMNA_2','NOMBRE_CAMPO']] = \
                                df_final_columns.CAMPOS.str.split(".", n=3, expand=True)

            for index, row in df_final_columns.iterrows(): 
                if ((row["NOMBRE_CAMPO"] == None or row["NOMBRE_CAMPO"] == np.NaN) \
                and (row["COLUMNA_2"]    == None or row["COLUMNA_2"]    == np.NaN) ):
                    row["NOMBRE_CAMPO"] = row["COLUMNA_1"]                 

                elif (row["NOMBRE_CAMPO"] == None or row["NOMBRE_CAMPO"] == np.NaN ):
                    row["NOMBRE_CAMPO"] = row["COLUMNA_2"] 
                    row["ALIAS_TMP"]    = row["COLUMNA_1"] 


        # Para los casos que viene el campo en formato: alias.campo    
        if (len(df_tmp_columns.columns) == 2):    
            df_final_columns[['ALIAS_TMP','NOMBRE_CAMPO']] = df_final_columns.CAMPOS.str.split(".", n=2, expand=True)

            for index, row in df_final_columns.iterrows(): 
                if (row["NOMBRE_CAMPO"] == None or row["NOMBRE_CAMPO"] == np.NaN ):
                    row["NOMBRE_CAMPO"] = row["ALIAS_TMP"] 
                    row["ALIAS_TMP"]    = 'N/A'


        # Para los casos que viene el campo en formato: campo   
        elif (len(df_tmp_columns.columns) == 1): 
            df_final_columns['ALIAS_TMP']  = 'N/A'
            df_final_columns['NOMBRE_CAMPO'] = df_final_columns.CAMPOS.str.split(".", n=1, expand=True)


            
        df_final_columns['ALIAS_TMP']  = df_final_columns['ALIAS_TMP'].str.upper() 
        df_final_tables['ALIAS_TABLA'] = df_final_tables['ALIAS_TABLA'].str.upper() 

        # Merge para los match completos entre campos y tablas (INNER JOIN)  
        df_merge = pd.merge(df_final_columns, df_final_tables, \
                            left_on  =["MULTINIVEL","ALIAS_TMP"], right_on =["MULTINIVEL","ALIAS_TABLA"], \
                            how ='outer')

        
        # Los casos en que las subquery tiene una sola tabla y los campos no tienen esquema, asignar esa tabla
        # Cuento la cantidad de tablas que tiene un multinivel
        df_count_tables = df_final_tables.groupby(['MULTINIVEL'])['ESQUEMA_INPUT','TABLA_INPUT'].count()

        # Obtengo los multiniveles que solo tienen 1 tabla 
        df_count_tables = df_count_tables[(df_count_tables['ESQUEMA_INPUT'] == 1) \
                                        & (df_count_tables['TABLA_INPUT'] == 1)].reset_index()
        df_count_tables = df_count_tables[['MULTINIVEL']]

        # Hago merge con el dataframe anterior para obtener de nuevo los nombres
        df_merge_count = pd.merge(df_final_tables, df_count_tables, on  =["MULTINIVEL"], how ='inner')


        # Asigna a los campos que no tienen match, su tabla correspondiente, en los casos de 1 sola tabla
        for index, row in df_merge_count.iterrows(): 
            df_merge['ESQUEMA_INPUT'] = np.where((df_merge['MULTINIVEL'] == row['MULTINIVEL']) & \
                                                (df_merge['ALIAS_TMP'] == 'N/A'), \
                                                row["ESQUEMA_INPUT"], df_merge["ESQUEMA_INPUT"] )        

            df_merge['TABLA_INPUT'] = np.where((df_merge['MULTINIVEL'] == row['MULTINIVEL']) & \
                                              (df_merge['ALIAS_TMP'] == 'N/A'), \
                                              row["TABLA_INPUT"], df_merge["TABLA_INPUT"] )

            df_merge['ALIAS_TABLA'] = np.where((df_merge['MULTINIVEL'] == row['MULTINIVEL']) & \
                                           (df_merge['ALIAS_TMP'] == 'N/A'), \
                                           row["ALIAS_TABLA"], df_merge["ALIAS_TABLA"] )

            df_merge['ALIAS_SUBQUERY'] = np.where((df_merge['MULTINIVEL'] == row['MULTINIVEL']) & \
                                           (df_merge['ALIAS_TMP'] == 'N/A'), \
                                           row["ALIAS_SUBQUERY"], df_merge["ALIAS_SUBQUERY"] )


        # Genero un campo compuesto con el formato esquema.tabla.campo 
        cols = ['ESQUEMA_INPUT', 'TABLA_INPUT', 'NOMBRE_CAMPO']
        df_merge['CAMPO_COMPUESTO'] = df_merge[cols].apply(lambda row: np.nan \
                    if (row['NOMBRE_CAMPO'] is np.nan or row['NOMBRE_CAMPO'] == None) \
                    else '.'.join(row.values.astype(str)), axis=1)

        # Obtego las columnas que quiero del df 
        df_merge_final = df_merge[columnsDF] 

    
    return df_merge_final
    

In [10492]:
def insertDataframeTablesQuery(df_total_tables, multilevel, table_list, subquery_alias):
    '''
    Función que inserta en un dataframe las tablas de una query
    '''
    new_df_tables       = emptyDataframeTablesQuery(None)
    
    #print (" ****** LISTA TABLAS  -> "          + str(table_list)) 

    ## SECCION DE INSERCION DE TABLAS SOLAMENTE
    for item in table_list:
        tamañoItem = len(item)
        
        if (tamañoItem == 3):
            new_df_tables = new_df_tables.append(
                [ {'MULTINIVEL': multilevel ,
                   'ESQUEMA_INPUT': item[0],
                   'TABLA_INPUT': item[1], 
                   'ALIAS_TABLA' : item[2] }], 
                    ignore_index=True, sort=False) 
                           
        elif (tamañoItem == 2):
            new_df_tables = new_df_tables.append(
                [ {'MULTINIVEL': multilevel ,
                   'ESQUEMA_INPUT': item[0],
                   'TABLA_INPUT': item[1] }], 
                    ignore_index=True, sort=False) 
        
    
    new_df_tables = new_df_tables.assign( ALIAS_SUBQUERY = subquery_alias )   
    df_total_tables = df_total_tables.append(new_df_tables)
        
    return df_total_tables

In [10493]:
def insertDataframeColumnsQuery(df_total_columns, multilevel, column_list, column_list_total,
            rule_select_list_total, rule_generic_list_total):
    '''
    Función que inserta en un dataframe los campos, alias, reglas y filtros de una query
    '''
    
    new_df_column       = emptyDataframeColumnsQuery(None)
    new_df_internal     = emptyDataframeColumnsQuery(None)
    
    # Lista los campos que tuvieron logica asociada
    check_rule_columns  = []
    
    # Lista temporal de campos y reglas
    list_columns_rules  = []
    alias               = ''
    flag_column_with_as = False  # Se activa si consigue que un campo contiene alias
    column_list = unique(column_list) 
    
    
    '''
    print (" ****** POSICION DE MULTINIVEL -> " + str(multilevel))
    print (" ****** LISTA CAMPOS  -> "          + str(column_list))
    print (" ****** LISTA CAMPOS TOTAL  -> "    + str(column_list_total))
    print (" ****** LISTA LOGICAS SELECT -> "   + str(rule_select_list_total)) 
    print (" ****** LISTA LOGICAS FILTROS -> "  + str(rule_generic_list_total)) 
    '''
    
    ## Recorre la lista de las reglas y si existe un campo que esté contenido en la regla
    ## entonces inserta en la misma fila el campo y la regla interna.      
    if (rule_select_list_total):
        
        ## Logicas de los 'select'
        for rule in rule_select_list_total:
            flag_column_rule = False
            
            # Recorre cada campo para hacer match con la regla/logica
            for column in column_list:                 
                if (column in rule[0] and column != rule[1]):                  
                    flag_column_rule = True
                    new_df_column = new_df_column.append(
                    [{  'MULTINIVEL': multilevel , 
                        'CAMPOS': column,
                        'LOGICA_NEGOCIO' : rule[0],
                        'ALIAS_CAMPO'   : rule[1]
                    }], 
                     ignore_index=True, sort=False)                 
                    check_rule_columns.append(column)
                    new_df_internal = new_df_internal.append(new_df_column,ignore_index=True, sort=False)
                    
                    
            ## Si la regla no tiene campo, insertarla de igual forma        
            if (flag_column_rule == False):
                new_df_column = new_df_column.append(
                [{  'MULTINIVEL': multilevel ,
                    'CAMPOS': None,
                    'LOGICA_NEGOCIO' : rule[0],
                    'ALIAS_CAMPO'   : rule[1]
                }], 
                 ignore_index=True, sort=False) 
                new_df_internal = new_df_internal.append(new_df_column,ignore_index=True, sort=False)
         
    ## Logicas de negocio de los filtros
    if (rule_generic_list_total):
        for rule in rule_generic_list_total:
            for column in column_list: 
                if (column in rule):
                    new_df_column = new_df_column.append(
                    [{  'MULTINIVEL': multilevel ,
                        'CAMPOS': column,
                        'LOGICA_NEGOCIO' : rule}], 
                     ignore_index=True, sort=False) 
                    
                    check_rule_columns.append(column)
                    new_df_internal = new_df_internal.append(new_df_column,ignore_index=True, sort=False)
            
    
    ## los campos solos que no tengan regla asociada, se insertan al final
    ## pero hay qe validar si tienen alias también
    columns_without_rules = list(set(column_list) - set(check_rule_columns))        
    
    
    if (columns_without_rules):
        for column_w_r in columns_without_rules:  
            flag_column_with_as = False
            
            if (column_list_total):
                for column_as in column_list_total:
                    
                    field = column_as[0]
                    len_list_field = len(column_as)
                    
                    if (len_list_field == 2):
                        if (field != None and column_w_r == field):
                            flag_column_with_as = True
                            alias = column_as[1]                            
                            new_df_column = new_df_column.append(
                                        [{  'MULTINIVEL': multilevel ,
                                            'CAMPOS': field,
                                            'ALIAS_CAMPO': alias 
                                         }], 
                                         ignore_index=True, sort=False)  
                            
            ## Si el campo de la lista de campos sin reglas no tiene alias, insertar al final
            if (flag_column_with_as == False):           
                new_df_column = new_df_column.append(
                            [{  'MULTINIVEL': multilevel ,
                                'CAMPOS': column_w_r,
                                'ALIAS_CAMPO': None 
                             }], 
                             ignore_index=True, sort=False) 
            
            new_df_internal = new_df_internal.append(new_df_column,ignore_index=True, sort=False)
    
            
    if (new_df_internal.empty == False):        
        new_df_internal  = new_df_internal.drop_duplicates()    
        new_df_internal  = new_df_internal.sort_values(by ='CAMPOS')
        df_total_columns = df_total_columns.append(new_df_internal,ignore_index=True, sort=False)
        
    
    return df_total_columns

In [10494]:
def updateInDataFrameColumnsRules(df_tables_columns, rule_generic_list_total):
    '''Función que actualiza o inserta en los campos las ultimas reglas encontradas en la query.
       Es decir, multiquery = 1 y antes del final con ; 
    '''
    last_rules    = []
    internal_rule = []
    
        
    if (rule_generic_list_total and df_tables_columns.empty == False):
        for rule in rule_generic_list_total: 
            
            # Si el campo No tiene lógica de negocio o regla, se actualiza el df
            df_tables_columns['LOGICA_NEGOCIO'] = \
                df_tables_columns.apply(lambda df: rule \
                   if (str(df['CAMPOS']) in rule and df['LOGICA_NEGOCIO'] is np.nan) \
                    else df['LOGICA_NEGOCIO'], axis=1)


            # Si el campo Si tiene lógica de negocio o regla diferente a la insertada,
            # se inserta el nuevo al df            
            for index, row in df_tables_columns.iterrows(): 
                if (row["MULTINIVEL"] == '1' and \
                    str(row["CAMPOS"]) in rule and \
                    row["LOGICA_NEGOCIO"] != rule ):

                    internal_rule = [row["MULTINIVEL"], row["CAMPOS"], rule]
                    last_rules.append(internal_rule)
            
    last_rules = unique(last_rules)    
    new_dataframe = pd.DataFrame(last_rules, columns = ['MULTINIVEL','CAMPOS','LOGICA_NEGOCIO'] )
    
    df_tables_columns = df_tables_columns.append(new_dataframe, ignore_index=True)
    
    return df_tables_columns

In [10495]:
def insertaEnArchivoDF(file, df):
    '''
    Función que inserta un dataframe a un archivo
    '''
    df.to_csv(file, header=True, index=None, sep='|', mode='w')

In [10496]:
def get_tables_output(query):
    ''' 
    Recorre los tokens de 'INSERT, REPLACE VIEW' hasta conseguir un 'SELECT'
    Retorna lista: [tipo de stmt, esquema output, tabla output]
    '''
    list_output = []
    create_seen         = False
    replace_seen        = False
    rename_seen         = False
    drop_seen           = False
    last_token          = None

    for token in sqllib.get_query_tokens(query):
        
        #print(str(token.ttype) + " --> " + str(token.value.upper()))
        
        if (token.ttype is DML and token.value.upper() == 'SELECT') \
        or (token.ttype is Punctuation and token.value in ['(',';']) \
        or (token.ttype is Keyword and token.value == 'VALUES') \
        or (len(list_output) == 3):
            
            #print(list_output)
            return list_output
            
        elif token.ttype is DML and \
            token.value.upper() in ['INSERT','UPDATE', 'DELETE'] :
            token_str = token.value.upper()
            list_output.append(token_str)
        
        elif (last_token == 'DELETE' and token.value.upper() != 'FROM' and len(token.value) <= 3):
            pass
        
        elif (last_token == 'UPDATE' and len(token.value) <= 3):
            pass
            
        elif token.ttype is DDL and token.value.upper() == 'CREATE':
            token_str = 'CREATE'
            create_seen = True

        elif token.is_keyword and token.value.upper() == 'RENAME':
            rename_seen = True            
  
        elif token.is_keyword and token.value.upper() == 'TABLE' and rename_seen:
            token_str = 'RENAME TABLE'
            list_output.append(token_str)            
            
        elif token.ttype is DDL and token.value.upper() == 'DROP':
            token_str = 'DROP'
            drop_seen = True
            list_output.append(token_str)

        elif token.is_keyword and token.value.upper() == 'JOIN' and drop_seen:
            token_str = 'DROP JOIN INDEX'
            list_output[-1] = token_str
        
        elif token.is_keyword and token.value.upper() == 'INDEX' and drop_seen:
            token_str = token_str + ' INDEX'
            list_output[-1] = token_str
            
        elif token.is_keyword and token.value.upper() == 'INDEX' and create_seen:
            token_str = token_str + ' INDEX'
            list_output.append(token_str)
 
        elif token.is_keyword and token.value.upper() == 'JOIN' and create_seen:
            token_str = 'CREATE JOIN INDEX'
            create_seen = False
            list_output.append(token_str)

        elif token.ttype is Name and token.value.upper() == 'SET' and create_seen:
            token_str = 'CREATE SET TABLE'
            create_seen = False
            list_output.append(token_str)
            
        elif token.ttype is Name and token.value.upper() == 'MULTISET' and create_seen:
            token_str = 'CREATE MULTISET TABLE'
            create_seen = False
            list_output.append(token_str)
            
        elif token.is_keyword and token.value.upper() == 'TABLE' and create_seen:
            token_str = 'CREATE TABLE'
            list_output.append(token_str)

        elif token.is_keyword and token.value.upper() == 'VIEW' and create_seen:
            token_str = 'CREATE VIEW'
            list_output.append(token_str)
        
        elif token.ttype is DML and token.value.upper() == 'REPLACE':
            replace_seen = True
            
        elif token.is_keyword and token.value.upper() == 'VIEW' and replace_seen:
            token_str = 'REPLACE VIEW'
            list_output.append(token_str)
        
        elif token.ttype is Name and token.value == 'HASH':
            token_str = token_str + ' HASH '
        
        ## Aqui va agregando la info. de las tablas output Formato [DDL, ESQUEMA, TABLA]
        elif token.ttype is Name and token.value not in ['HASH']:
            list_output.append(token.value)
            
        
        last_token = token.value.upper() 
                   
    return 'DESCONOCIDO'    

In [10497]:
def get_output_input_tables(folder, filename, code_lines, queryString, process_number):
    '''
    Función que procesa una query retornando su tabla output y la lista de tablas input
    Si no existen tablas input, se coloca un mensaje que indica que no tiene condiciones
    '''
    
    list_table_input  = []
    list_table_output = []
    columns = [ 'TIPO_ARCHIVO','ARCHIVO','LINEAS_DE_CODIGO','NUMERO_PASO','SENTENCIA_DML',
               'ESQUEMA_OUTPUT','TABLA_OUTPUT','MULTINIVEL','ESQUEMA_INPUT','TABLA_INPUT',
               'ALIAS_SUBQUERY', 'COLUMNA','VARIABLE','LOGICA_NEGOCIO', 'ALIAS_CAMPO']
    
    df_final_merge       = emptyDataframeExtractQuery(folder)
    df_merge_table_input = emptyDataframeExtractQuery(folder)

    list_syntax_large = ['CREATE MULTISET TABLE', 'DROP JOIN INDEX', 'CREATE INDEX']
    
    if (queryString):

        parsed                  = sqlparse.parse(queryString)[0]
        type_DML_DDL_principal  = parsed.token_first().value.upper()
        list_table_output       = get_tables_output(queryString)
        

        if(list_table_output[0] not in list_syntax_large):
                
            if (type_DML_DDL_principal in ['DELETE', 'UPDATE']):

                df_merge_table_input = get_query_tables_and_columns(queryString)
                

            elif (list_table_output[0] in ['DROP']):
                output_table    = sqllib.get_query_tables(queryString)[0] 
                l_output_table  = output_table.split('.')
                len_l_output_table = len(l_output_table)                
                
                # Para los casos que hay tablas volatiles sin esquema
                if (len_l_output_table == 2):
                    database    = l_output_table[0]
                    table       = l_output_table[1]
                    
                    list_table_output.clear() 
                    list_table_output.append(type_DML_DDL_principal)
                    list_table_output.append(database) 
                    list_table_output.append(table) 
                
                elif (len_l_output_table == 1):
                    table       = l_output_table[0]
                    
                    list_table_output.clear() 
                    list_table_output.append(type_DML_DDL_principal)
                    list_table_output.append('N/A') 
                    list_table_output.append(table) 
                
            else:
                df_merge_table_input = get_query_tables_and_columns(queryString)

    ## Si no tiene tablas, agregar info de no tener condicion de tablas input
    #print(df_merge_table_input)
    
    if (df_merge_table_input.empty): 
        df_table_input = pd.DataFrame({"MULTINIVEL": ['0'],
                                       "ESQUEMA_INPUT":[noConditionInput], 
                                       "TABLA_INPUT":[noConditionInput]})
        
        df_merge_table_input = df_merge_table_input.append(df_table_input, sort=True)
        
        

    df_merge_table_input = df_merge_table_input.rename(columns={
                                                    'NOMBRE_CAMPO'   : 'COLUMNA',
                                                    'CAMPO_COMPUESTO': 'VARIABLE'})
        
    df_final_merge = df_final_merge.append(df_merge_table_input, sort=True)
    

    if (len(list_table_output) >= 3):
        # Asigna los valores constantes a dicho dataframe
        if(folder != 'Vistas'):
            df_final_merge = df_final_merge.assign(TIPO_ARCHIVO = folder, \
                                                   ARCHIVO = filename, \
                                                   LINEAS_DE_CODIGO = code_lines, \
                                                   NUMERO_PASO = process_number, \
                                                   SENTENCIA_DML  = list_table_output[0], \
                                                   ESQUEMA_OUTPUT = list_table_output[1], \
                                                   TABLA_OUTPUT   = list_table_output[2])
        else:
            df_final_merge = df_final_merge.assign(TIPO_ARCHIVO = folder, \
                                                   ARCHIVO = filename, \
                                                   LINEAS_DE_CODIGO = code_lines, \
                                                   SENTENCIA_DML  = list_table_output[0], \
                                                   ESQUEMA_OUTPUT = list_table_output[1], \
                                                   TABLA_OUTPUT   = list_table_output[2])
    
    # Reordenando los campos del dataframe 
    df_final_merge = df_final_merge.replace(to_replace='nan.', value='', regex=True)
    df_final_merge = df_final_merge.replace(to_replace='nan',  value='', regex=True)
    df_final_merge = df_final_merge.reindex(columns , axis=1)
    
    return df_final_merge

In [10498]:
def generate_number_by_list(list_unique_str_num_query, last_number_subquery):
    ''' 
    Función que crea el multinivel de las queries, 
    con secuencia: 1.1, 1.2, 1.2.1, según lista de multiveles ya creados
    '''    
    list_final = []
    
    lenght_list       = len(list_unique_str_num_query)
    length_last_query = len(last_number_subquery)
    
    if (length_last_query == 1 and lenght_list > 1):        
        for item in list_unique_str_num_query:
            
            if (len(item) == 3):
                first_item = item.split('.')[0]
                
                if (first_item == last_number_subquery):
                    list_final.append(item)
                
        if list_final:
            last_multilevel      = max(list_final)
            split                = last_multilevel[:-1]
            last_number          = last_multilevel[-1]
            next_number          = int(last_number) + 1
            new_number_subquery  = split + str(next_number)
        else:
            new_number_subquery  = last_number_subquery + '.1'
        
    elif (length_last_query >= 3 and lenght_list > 1):      
        for item in list_unique_str_num_query:
            if (len(item) == length_last_query + 2 and \
               last_number_subquery in item):
                list_final.append(item)
        
        if list_final:                
            last_multilevel      = max(list_final)
            split                = last_multilevel[:-1]
            last_number          = last_multilevel[-1]
            next_number          = int(last_number) + 1
            new_number_subquery  = split + str(next_number)
        else:
            new_number_subquery  = last_number_subquery + '.1'
                
   
        
    else:
        new_number_subquery  = '1.1'   
        
        
    return new_number_subquery

In [10499]:

def generateNumberedListSubquery(flag_get_in_subquery, flag_lastjoin_is_subquery,
                                 flag_union_join_query,count_query_multilevel, 
                                 last_number_subquery, list_unique_str_num_query):
    
    ''' 
    Función que crea el multinivel de las queries, con secuencia: 1.1, 1.2, 1.2.1, etc
    '''
    
    point = '.'
    new_number_subquery = None
    entered_in_other_rule = False
    length_last_query = len(last_number_subquery)
    
    '''
    print("last_number_subquery      ---> "+ last_number_subquery + ", length_last_query ---> "+ str(length_last_query))
    print("count_query_multilevel    ---> "+ str(count_query_multilevel))
    print("flag_get_in_subquery      ---> "+ str(flag_get_in_subquery))
    print("flag_lastjoin_is_subquery ---> "+ str(flag_lastjoin_is_subquery))
    print("flag_union_join_query     ---> "+ str(flag_union_join_query))
    '''
    
    '''Sección para los casos de UNION'''
    if (flag_union_join_query and flag_get_in_subquery == False \
    and flag_lastjoin_is_subquery == False):
        ## Si es = 1 entonces ponerle .1 , sino es sumarle 1 al ultimo valor        
        if (length_last_query == 1):
            new_number_subquery = str(int(last_number_subquery) + 1)
        else:
            split               = last_number_subquery[:-1]
            next_number         = int(last_number_subquery[-1]) + 1
            new_number_subquery = split + str(next_number)

            
    elif (flag_union_join_query == False):
        
        '''Sección donde entra a las subqueries'''
        ## Abre parentesis y consigue la primera subquery
        if (flag_lastjoin_is_subquery == False):

            if (length_last_query in [1,2] and flag_get_in_subquery):
                entered_in_other_rule = True
                new_number_subquery = generate_number_by_list(list_unique_str_num_query, last_number_subquery)

            elif (length_last_query >= 3 and flag_get_in_subquery):
                split               = last_number_subquery[:-1]
                next_number         = int(last_number_subquery[-1]) + 1
                new_number_subquery = split + str(next_number)

            elif (length_last_query == count_query_multilevel):
                new_number_subquery = str(last_number_subquery) + '.1'

        '''Sección donde sale de las subqueries'''     
        if (length_last_query >= 3 and flag_get_in_subquery == False):
            # Se elimina el ultimo numero del multinivel para volver al principal del mismo
            number_subquery_list = last_number_subquery.split('.')
            number_subquery_list = number_subquery_list[:-1]
            
            for item in number_subquery_list:
                if new_number_subquery == None:
                    new_number_subquery = item
                else:
                    new_number_subquery = new_number_subquery + '.' + item
        
            entered_in_other_rule = True

        elif (flag_lastjoin_is_subquery):
            new_number_subquery = generate_number_by_list(list_unique_str_num_query, last_number_subquery)
        

        if (length_last_query == count_query_multilevel and entered_in_other_rule == False):
            new_number_subquery = str(last_number_subquery) + '.1'   
            

    return new_number_subquery
    

In [10500]:
def get_query_tables_and_columns(query):
    """
    Función que obtiene todos los objetos de una query(tablas, campos, reglas) 
    
    Descripción: Por cada nivel de subquery que consiga, inserta en un dataframe los 
    objetos que consiga según ese nivel.
    """    
    
    '''LISTAS Y MANEJO DE OBJETOS (TABLAS, CAMPOS Y REGLAS)'''
    
    
    column                    = None # Contiene el último valor de un campo del ´SELECT´
    
    str_rule_agg              = '' # Cadena que va agregando la regla a medida que avanza el token
    
    tables                    = [] # Lista interna de tabla con formato [ESQUEMA, TABLA, ALIAS]
    
    column_list               = [] # Lista de columnas con formato [ESQUEMA.CAMPO1, ALIAS_1]
    
    column_list_internal      = [] # Lista de columnas con formato [ESQUEMA.CAMPO1, ALIAS_1]
    
    column_list_total         = [] # Lista total de todas las columnas con formato [ESQUEMA.CAMPO1, ALIAS_1]  
                                   # Ej: [[ESQUEMA.CAMPO1, ALIAS_1],[ESQUEMA.CAMPO2, ALIAS_2] ]
    
    table_list                = [] # Lista interna de tabla con formato [ESQUEMA, TABLA, ALIAS] 
    
    table_list_total          = [] # Lista total de todas las tablas encontradas en una query o subquery.     
                                   # Ej: [[ESQUEMA1, TABLA1, ALIAS1],[ESQUEMA1, TABLA2, ALIAS2] ]
                                   
    rule_select_list          = [] # Lista interna de una regla o función encontrada en un SELECT.
                                   # Con formato [str_rule_agg, column_alias]
                                   # Ej: [MAX(COLUMN), ALIAS]
    
    rule_select_list_total    = [] # Lista total de todas las reglas y funciones encontradas en un SELECT
                                   # Con formato [[str_rule_agg, column_alias],[str_rule_agg, column_alias]]
                                   # Ej: [[MAX(CAMPO_1), ALIAS_1], [SUBSTR(CAMPO_2), ALIAS_2]]
    
    rule_generic_list_total   = [] # Lista total de todas las reglas encontradas en un ´JOIN´ o ´WHERE´
    
    
    '''FLAGS PARA EL CONTROL DE LAS REGLAS INTERNAS'''
    flag_rule_select           = False  # Se activa si consigue una funcion como regla [functions_sql]
    flag_rule_filter           = False  # Se activa si consigue un ´ON´ o ´WHERE´
    flag_end_rule_as           = False  # Se activa si es el final de una regla con un ALIAS cuando consigue ´AS´
    flag_rule_case             = False  # Se activa si es el final de una regla de CASE cuando consigue ´END´ o ´AS´
    flag_rule_case_internal    = False  # Se activa si es una regla de CASE dentro de otra función
    flag_rule_in_before_select = False  # Se activa si consigue '(' después de un 'IN'
        
    flag_open_parentheses_rule = False  # Se activa cuando estoy dentro de una regla y consigue un parantesis ´(´
    flag_count_parentheses_rule = False # Se activa cuando los parentesis abiertos y cerrados son iguales en una regla
    
    
    '''MANEJO DEL TOKEN DE QUERY'''
    last_keyword     = None     # Contiene el último valor del Keyword (palabra reservada SQL) del token 
    last_token       = None     # Contiene el último valor del token
    subquery_alias   = None     # Contiene el ALIAS de una subquery
    column_alias     = None     # Contiene el ALIAS de una subquery
    
    
    
    '''VARIABLES PARA EXTRAER LAS TABLAS INPUT'''
    dml_ddl_name      = None
    not_get_from      = False
    from_clausule     = False
    flag_create_table = False
    
        
    '''DATAFRAMES'''    
    df_final_tables             = emptyDataframeTablesQuery(None)
    df_internal_table           = emptyDataframeTablesQuery(None)
    
    df_final_columns            = emptyDataframeColumnsQuery(None)
    df_internal_column          = emptyDataframeColumnsQuery(None)
    
    df_merge_table_column_input = emptyDataframeTablesColumnsInputQuery(None)
    
    
    '''MULTINIVEL DE SUBQUERIES'''
    flag_open_parentheses_subq  = False  # Se activa cuando se abre parentesis por una subquery
    flag_close_parentheses_subq = False  # Se activa cuando se cierra parentesis por una subquery
    flag_is_subquery            = False  # Se activa cuando consigue una subquery
    flag_lastjoin_is_subquery   = False  # Se activa cuando hay una subquery dentro de otra subquery
    num_query                   = 0      # Nivel del select o subquery.
    
    '''CONTADORES DE PARENTESIS PARA SABER CUANDO EMPIEZA O TERMINA UNA QUERY'''
    count_query_multilevel     = 0  
    count_open_parentheses     = 0        # Contador de parentesis '(' externos 
    count_close_parentheses    = 0        # Contador de parentesis ')' externos 
    count_open_parentheses_rule  = 0      # Contador de parentesis '(' de reglas 
    count_close_parentheses_rule = 0      # Contador de parentesis ')' de reglas
    count_open_parentheses_internal  = 0  # Contador de parentesis '(' internos de una subquery
    count_close_parentheses_internal = 0  # Contador de parentesis ')' internos de una subquery
    
    
    list_unique_str_num_query  = set({'1'})
    str_num_query              = '1'
    last_number_subquery       = '1'
    
    
    cast_data_type_list = ['INTEGER', 'CHAR', 'DECIMAL', 'VARCHAR','TIMESTAMP']
        
    stop_from_list = ['AS','CASE','WHEN','ON','AND','END','WHERE','GROUP','OVER','PARTITION','SET']
    
    functions_ignored = ['COUNT', 'MIN', 'MAX', 'SUM', 'FROM_UNIXTIME', 'DEC', 'QUALIFY',
    'CAST', 'CONVERT', 'ZEROIFNULL','SUBSTR','SUBSTRING','ROW_NUMBER', 'RANK', 'ADD_MONTHS',
    'COALESCE', 'CHAR', 'INTEGER', 'TRIM', 'OVER', 'FORMAT', 'DATE_FORMAT','CHARACTER_LENGTH',
    'CHAR_LENGTH']
    
    dates_list = ['SECOND', 'MINUTE', 'HOUR', 'DAY', 'YEAR','MONTH','BOTH', 'TRAILING']
    
    columns_syntax_keywords = ['CURRENT_TIMESTAMP','CURRENT_TIME']
    
    # Lista de palabras reservadas que DEBEN ser consideras al obtener una regla.
    keywords_ignored = ['AS', 'AND', 'OR', 'IN', 'IS', 'NOT','NOT NULL', 'NULL',
    'LIKE', 'CASE', 'WHEN', 'ON', 'CURRENT_DATE']

    # Lista de funciones reservadas de SQL ANSI
    functions_sql = ['AVG', 'TITLE', 'OVER', 'OREPLACE','COUNT', 'MIN', 'MAX', 'SUM', 
    'FROM_UNIXTIME', 'DEC', 'CAST', 'CONVERT','ZEROIFNULL','SUBSTR','ROW_NUMBER','RANK',
    'COALESCE', 'CHAR', 'TRIM', 'FORMAT', 'EXTRACT','ADD_MONTHS','DISTINCT', 'QUALIFY', 
    'DATE_FORMAT', 'ELSE', 'LAST_DAY', 'SUBSTRING', 'UPPER', 'TRUNC','CHARACTER_LENGTH'
    ]
    # quité el INTEGER de aqui
    
    # Lista de funciones reservadas de SQL ANSI para saber que comienza una regla
    functions_rules = ['ON',  'WHERE']
    
    # Lista de funciones reservadas de SQL ANSI para saber que lo siguiente será un campo.
    functions_before_column = ['SEL', 'SELECT', 'ON', 'END', 'THEN', 'DISTINCT', 'WHERE']
    
    # Lista de funciones reservadas de SQL ANSI para las uniones de tablas
    table_syntax_joins = ['FROM', 'JOIN', 'INNER JOIN', 'LEFT JOIN', 'LEFT OUTER JOIN', 
	'RIGHT JOIN', 'RIGHT OUTER JOIN','CROSS JOIN']
    
    union_joins_list = ['UNION', 'UNION ALL']
        
   
    # Lista de sintaxis reservadas de SQL.     
    table_syntax_keywords = [
        # SELECT queries
        'FROM', 'WHERE', 'JOIN', 'INNER JOIN', 'LEFT JOIN', 'LEFT OUTER JOIN', 
		'RIGHT JOIN', 'RIGHT OUTER JOIN', 'ON', 'UNION', 'UNION ALL','CROSS JOIN',
        # INSERT queries
        'INTO', 'VALUES',
        # UPDATE queries
        'UPDATE', 'SET',
        # Hive queries
        'TABLE',  # INSERT TABLE
    ]

    for token in sqllib.get_query_tokens(query):
            
        
        ## Homologar en caso que se consiga un 'SEL' en vez de 'SELECT'
        if ((token.ttype is Name and token.value.upper() == 'SEL')):
            token.ttype = DML
            token.value = 'SELECT' 
        
        if (count_open_parentheses > 0):
            flag_is_subquery = True
        

        
        ## Es una regla de IN con Subquery y no debe cortar el proceso.
        if (token.value == '('):
            if (last_token in ['IN', 'EXISTS', 'NOT EXISTS']):
                flag_rule_in_before_select = True
                    
        
        ## Cálculo de multiniveles de queries 
        if (flag_is_subquery):
            if (token.value == '('):
                count_open_parentheses += 1
                    
            elif (token.value == ')'):
                count_close_parentheses += 1
        
        
        ## Verifica si despues del IN viene un SELECT
        if (flag_rule_in_before_select):
            if(token.value == 'SELECT' and last_token == '('):
                flag_rule_in_before_select = True
            else:
                flag_rule_in_before_select = False
        
        
        
        ## Una vez se consiga el fin de la subquery o UNION de querys, inserta en un df sus caracteristicas
        if (token.ttype is Name and last_token in [')', 'AS']): 
            
            ## Si los parentesis externos son iguales, es porque se cerró la subquery grande
            if (count_open_parentheses == count_close_parentheses and \
                count_open_parentheses > 0):
                
                
                # Obtiene el alias de la subquery
                subquery_alias = token.value
 
                # Obtiene el ultimo filtro que esté acumulado de la subquery
                if (flag_rule_filter or flag_rule_select):
                    flag_rule_filter  = False
                    flag_rule_select  = False
                    str_rule_agg = str_rule_agg.replace(") "+ last_token, '')
                    rule_generic_list_total.append(str_rule_agg)
                    str_rule_agg = ''
                
                
                df_internal_table = insertDataframeTablesQuery(
                                    df_internal_table, str_num_query,
                                    table_list_total,subquery_alias ) 
                
                df_final_tables   = df_final_tables.append(df_internal_table, ignore_index = True) 
                
                
                # Acumulación de objetos de campos y reglas internas
                df_internal_column = insertDataframeColumnsQuery(
                                    df_internal_column, str_num_query,
                                    column_list, column_list_total, rule_select_list_total,
                                    rule_generic_list_total)            
                
                df_final_columns = df_final_columns.append(df_internal_column, ignore_index = True) 

    
                ## limpia los objetos en cada nivel de subselect
                df_internal_table            = emptyDataframeTablesQuery(None)
                df_internal_column           = emptyDataframeColumnsQuery(None)
                column_list_total            = []                
                column_list_internal         = []
                table_list_total             = []
                column_list                  = []
                table_list                   = []
                rule_select_list_total       = []
                rule_generic_list_total      = []
                column_alias = None
                
        
                
                '''   
                print("count_open_parentheses  = "+ str(count_open_parentheses))
                print("count_close_parentheses = "+ str(count_close_parentheses))
                print("count_open_parentheses_internal  = " + str(count_open_parentheses_internal))
                print("count_close_parentheses_internal = " + str(count_close_parentheses_internal))
                print("flag_open_parentheses_subq = " + str(flag_open_parentheses_subq))
                
                ''' 
                
                flag_is_subquery            = False 
                not_get_from                = False
                flag_rule_in                = False                
                flag_rule_case              = False
                flag_rule_case_internal     = False
                flag_open_parentheses_subq  = False   
                flag_close_parentheses_subq = True
                count_open_parentheses      = 0
                count_close_parentheses     = 0  
                count_open_parentheses_rule  = 0
                count_close_parentheses_rule = 0
                
                    
                str_num_query = generateNumberedListSubquery(False,
                                                             flag_lastjoin_is_subquery,
                                                             False,
                                                             count_query_multilevel,
                                                             str_num_query,
                                                             list_unique_str_num_query) 
                
                if (str_num_query == None):
                    str_num_query = '1'
                    
                last_number_subquery  =   str_num_query 
                list_unique_str_num_query.add(str_num_query)
                
            
                
        ## Cálculo de multiniveles de queries internas (subquery dentro de subquery)
        if (flag_lastjoin_is_subquery and len(str_num_query) > 3 and flag_open_parentheses_subq):
            
            if (token.value == '('):
                count_open_parentheses_internal += 1
            elif (token.value == ')'):
                count_close_parentheses_internal += 1
         
            if (token.ttype is Name and last_token == ')'):   
                if ((count_open_parentheses_internal == count_close_parentheses_internal and \
                    count_open_parentheses_internal > 0 and flag_open_parentheses_subq) or \
                    (count_open_parentheses-1 == count_close_parentheses)):
                                       
                    # Obtiene el ultimo filtro que esté acumulado de la subquery
                    if (flag_rule_filter and last_keyword != 'ROW_NUMBER'):
                        flag_rule_filter = False
                        str_rule_agg = str_rule_agg.replace(") "+ last_token, '')
                        rule_generic_list_total.append(str_rule_agg)
                        str_rule_agg = ''
                    
                    
                    # Acumulación de objetos de tablas
                    df_internal_table = insertDataframeTablesQuery(
                                        df_internal_table, str_num_query,
                                        table_list_total,subquery_alias ) 

                    df_final_tables   = df_final_tables.append(df_internal_table, ignore_index = True) 


                    # Acumulación de objetos de campos y reglas internas
                    df_internal_column = insertDataframeColumnsQuery(
                                        df_internal_column, str_num_query,
                                        column_list, column_list_total, rule_select_list_total,
                                        rule_generic_list_total)            

                    df_final_columns = df_final_columns.append(df_internal_column, ignore_index = True) 


                    ## limpia los objetos en cada nivel de subselect
                    df_internal_table            = emptyDataframeTablesQuery(None)
                    df_internal_column           = emptyDataframeColumnsQuery(None)
                    column_list_total            = []
                    column_list_internal         = []
                    table_list_total             = []
                    column_list                  = []
                    table_list                   = []
                    rule_select_list_total       = []
                    rule_generic_list_total      = []
                    flag_rule_filter             = False              
                    flag_rule_case               = False
                    flag_rule_case_internal      = False
                    not_get_from                 = False
                    
                    
                    '''
                    print("count_open_parentheses  = "+ str(count_open_parentheses))
                    print("count_close_parentheses = "+ str(count_close_parentheses))
                    print("count_open_parentheses_internal  = " + str(count_open_parentheses_internal))
                    print("count_close_parentheses_internal = " + str(count_close_parentheses_internal))
                    print("flag_open_parentheses_subq = " + str(flag_open_parentheses_subq))
                    '''
                    
                    count_open_parentheses_internal  = 0
                    count_close_parentheses_internal = 0  
                    count_open_parentheses_rule  = 0
                    count_close_parentheses_rule = 0
                    flag_close_parentheses_subq      = True

                    
                    str_num_query = generateNumberedListSubquery(False,
                                                                 flag_lastjoin_is_subquery,
                                                                 False,
                                                                 count_query_multilevel,
                                                                 str_num_query,
                                                                 list_unique_str_num_query) 

                    
                    last_number_subquery  =   str_num_query 
                    list_unique_str_num_query.add(str_num_query)
  
            
        ## Se insertan los ultimos campos y tablas encontrados, antes de un subselect o fin de la query.
        if ((token.ttype is DML and token.value.upper() == 'SELECT'\
          or token.ttype is Punctuation and token.value == ';') and \
            flag_rule_in_before_select == False and \
            flag_rule_select == False and \
           (last_token not in functions_rules or last_keyword not in functions_rules)):       
                
            if (num_query > 0):  
                
                # Para los casos que vengan consultas con UNION
                if (last_token in union_joins_list):
                    last_number_subquery = str_num_query
                    str_num_query = generateNumberedListSubquery(False,
                                                                 False,
                                                                 True,
                                                                 count_query_multilevel,
                                                                 str_num_query,
                                                                 list_unique_str_num_query) 
                
                # Calcular el multinivel siempre que entra a la subquery
                if (last_token == '(' and last_keyword in table_syntax_joins):
                                    
                    flag_is_subquery = True
                    flag_open_parentheses_subq = True
                        
                        
                    if(len(str_num_query) >= 3):
                        flag_lastjoin_is_subquery = True
                    
                    if(count_open_parentheses == 0):
                        count_open_parentheses += 1
                        
                    count_query_multilevel += 1
                    
                    ## Comienza contador de parentesis interno de la subquery
                    if (flag_lastjoin_is_subquery and len(str_num_query) > 1):
                        
                        ## Query de más afuera donde sería la principal y acumula esos parentesis
                        if (len(str_num_query) > 3):
                            count_open_parentheses_internal  = 0
                            count_close_parentheses_internal = 0 
                        
                        count_open_parentheses_internal += 1
                    
                    last_number_subquery  =   str_num_query   
                    '''
                    print("EL ESTADO DE LOS PARENTESIS ES = ")
                    print("count_open_parentheses  = "+ str(count_open_parentheses))
                    print("count_close_parentheses = "+ str(count_close_parentheses))
                    print("ANTES DE ENTRAR generateNumberedListSubquery 3")
                    '''
                    str_num_query = generateNumberedListSubquery(True,
                                                                 flag_lastjoin_is_subquery,
                                                                 False,
                                                                 count_query_multilevel,
                                                                 str_num_query,
                                                                 list_unique_str_num_query) 
                   
                    
                    list_unique_str_num_query.add(str_num_query)
                    
            if (last_token not in functions_rules):
                last_keyword = 'SELECT'  
           
                
            num_query += 1            
            #Omite el 'SELECT' principal
            if (num_query >= 2):                
 
                # Obtiene el ultimo filtro que esté acumulado de la subquery
                if (token.ttype is Punctuation and token.value == ';' and flag_rule_filter):
                    flag_rule_filter  = False
                    flag_rule_select  = False
                    rule_generic_list_total.append(str_rule_agg)
                    str_rule_agg = ''
                    
                    
                # Acumulación de objetos de tablas
                df_internal_table = insertDataframeTablesQuery(
                                    df_internal_table, last_number_subquery,
                                    table_list_total,subquery_alias ) 
                
                df_final_tables   = df_final_tables.append(df_internal_table, ignore_index = True) 
                
                
                # Acumulación de objetos de campos y reglas internas
                df_internal_column = insertDataframeColumnsQuery(
                                    df_internal_column, last_number_subquery,
                                    column_list, column_list_total, rule_select_list_total,
                                    rule_generic_list_total)            
                
                df_final_columns = df_final_columns.append(df_internal_column, ignore_index = True) 

    
                ## limpia los objetos en cada nivel de subselect
                df_internal_table            = emptyDataframeTablesQuery(None)
                df_internal_column           = emptyDataframeColumnsQuery(None)
                column_list_total            = []                
                table_list_total             = []
                column_list_internal         = []
                column_list                  = []
                table_list                   = []
                rule_select_list_total       = []
                rule_generic_list_total      = []
                str_rule_agg = ''
                flag_is_subquery             = True
                flag_rule_filter             = False
                not_get_from                 = False
    
        #print(['1', '***  ' + str(str_num_query)  + '  ***', token, token.ttype, last_token, last_keyword])              
        
    
    ####################### EXTRACCIÓN DE LOGICAS DE CODIGO ########################################
        '''
        Sección que recupera las reglas internas y los filtros de una query,
        junto con sus alias respectivos
        Retorna una lista con el formato: [str_rule_agg, column_alias]
        A su vez si inserta en una lista        
        ''' 

        '''
        print("flag_is_subquery              = " + str(flag_is_subquery))
        print("not_get_from                  = " + str(not_get_from))
        print("flag_rule_filter              = " + str(flag_rule_filter))
        print("flag_rule_select              = " + str(flag_rule_select))
        print("flag_end_rule_as              = " + str(flag_end_rule_as))
        print("flag_rule_case                = " + str(flag_rule_case))
        print("flag_rule_case_internal       = " + str(flag_rule_case_internal)) 
        print("flag_rule_in_before_select    = " + str(flag_rule_in_before_select))       
        print("count_open_parentheses        = " + str(count_open_parentheses))
        print("count_close_parentheses       = " + str(count_close_parentheses))
        print("flag_open_parentheses_rule    = " + str(flag_open_parentheses_rule)+ "\n" )
        print("count_open_parentheses_rule   = " + str(count_open_parentheses_rule))
        print("count_close_parentheses_rule  = " + str(count_close_parentheses_rule)) 
        print("flag_count_parentheses_rule   = " + str(flag_count_parentheses_rule))             
        '''
        
        # Para los casos que toma el subquery_alias como column_alias y continua con algun JOIN
        token_value_clean = " ".join(token.value.upper().split())        
        if (token_value_clean in table_syntax_joins and \
            column_alias == last_token):
            #print("Quitar el alias de la regla = "+ column_alias)
            column_alias = None
            
        

        ##CASE, ON, FUNCTIONS_SQL   
        if ((token.value.upper() in keywords_ignored or token.value.upper() in functions_sql) \
            and flag_rule_filter == False 
            and flag_rule_select == False):
            
            flag_rule_case_internal = False            
            
            if (token.value.upper() in ['CASE']): 
                flag_rule_case   = True
                flag_rule_select = True                
                                               
            elif (token.value.upper() in ['ON','QUALIFY']):        
                flag_rule_filter = True  
                
            elif (token.value.upper() in functions_sql and flag_rule_filter == False):          
                flag_rule_select = True  
                
            elif (token.value.upper() == 'NULL'and (last_token == ',' or last_keyword == 'SELECT')):        
                flag_rule_select = True  
 
        elif (flag_rule_select and token.value.upper() in ['CASE']):  
            flag_rule_case_internal   = True 
            
            
        elif (token.value.upper() in ['CASE']):  
            flag_rule_case   = True 
        
        
        ## Casos de campos constantes. Ejemplo: 'Clientes Con Campana' (Varchar(50)) Descripcion 
        elif ((token.ttype is Literal.String.Single or \
               token.ttype is Literal.Number.Integer)
              and flag_rule_filter == False  \
              and flag_rule_select == False  \
              and (last_token == ',' or last_keyword == 'SELECT') ):    
            flag_rule_select = True  
            
        
        
        # Para concluir los casos de CAST ('' AS CHAR(40)) AS ALIAS_CAMPO
        # Y cualquier otro ALIAS de campos en general
        elif (token.is_keyword and token.value.upper() == 'AS' \
            and last_token in [')', 'END',"'"+'0000'+"'"] ):
        
            if (count_open_parentheses_rule == 0):
                flag_end_rule_as = True
            else:
                flag_end_rule_as = False
            
            if (last_token == 'END' and last_keyword == 'END' and count_open_parentheses_rule == 0):
                flag_rule_select = False                  
                flag_rule_case   = False
        
           
        ## No interrumpir la acumulacion de logica asi consiga ',' 
        ## Si no hay conseguido AS, si no es un filtro, si hay un parentesis abiertos '('
        elif ((token.ttype is Punctuation and token.value == ',' or \
              token.ttype is Name and token.value.upper() not in functions_sql) \
            and str_rule_agg != ''  \
            and flag_end_rule_as == False \
            and flag_rule_filter == False \
            and flag_open_parentheses_rule == False):
            

            if (flag_rule_case and last_keyword == 'END' \
            and flag_rule_case_internal == False):
                flag_rule_case = False
            
            
            # Para los casos por ejemplo: MAX(CASE WHEN RNK=1 THEN XCTA END) NTRJ1
            if (flag_rule_case_internal and last_keyword == 'END' \
            and count_open_parentheses_rule == 0):
                flag_rule_case_internal = False
            
            
            ## Fin de reglas en el SELECT
            if (flag_rule_case == False and flag_rule_case_internal == False
               and count_open_parentheses_rule == 0 ):
                
                if(token.ttype is Name and last_token in [')','AS'] \
                and last_keyword not in ['ANY','BY','DESC']):
                    column_alias = token.value.upper()
                
                flag_rule_select = False 
                rule_select_list = [str_rule_agg, column_alias]
                rule_select_list_total.append(rule_select_list)
                str_rule_agg = ''                
                count_open_parentheses_rule  = 0
                count_close_parentheses_rule = 0
                flag_count_parentheses_rule = False
                last_keyword = 'SELECT'
         
       ## Fin del ON o WHERE         
        elif ((flag_rule_filter and token.value.upper() in table_syntax_keywords \
              and token.value.upper() != 'FROM' and flag_rule_in_before_select == False)\
              or (token.ttype is Punctuation and token.value in [';'])):
            
                flag_rule_filter = False
                str_rule_agg = str_rule_agg.replace(") "+ last_token, '')
                rule_generic_list_total.append(str_rule_agg)
                str_rule_agg = ''
                count_open_parentheses_rule  = 0
                count_close_parentheses_rule = 0
                flag_count_parentheses_rule = False
                
        elif token.ttype is Name:
            
            ## Para los casos que el campo sea 'srvaprimo' Submit_User,
            if (str_rule_agg.strip() == last_token and \
                str_rule_agg.strip() not in functions_sql and \
                str_rule_agg.strip() not in functions_before_column and \
                str_rule_agg.strip() not in keywords_ignored and \
                flag_rule_filter == False):  
                column_alias = token.value                 
        
          
            ## Tomar el alias de la regla 
            if(flag_count_parentheses_rule and flag_rule_case == False \
            and token.value != subquery_alias \
            and (flag_rule_filter == False and flag_rule_select == False) \
            and last_token not in ['SELECT','AND','ON', '(', '.', '=', '+', '*',',']):
                
                #column_alias = last_token
                column_alias = token.value                
                flag_count_parentheses_rule = False                
                
            ## Fin del CASE con ALIAS y sin tener referencia de AS
            if (last_token == 'END' and last_keyword == 'END'):  
                last_token = 'AS'
                flag_end_rule_as = True
                flag_rule_case   = False
            
            
            ## Fin de la regla con ALIAS y con AS
            if (last_token == 'AS' and flag_count_parentheses_rule and token.value != subquery_alias):
                column_alias = token.value
                flag_end_rule_as = True
            
            
            ## Fin del CASE con ALIAS y con AS
            if (last_token == 'AS' and flag_end_rule_as and last_keyword != 'TABLE'):
                
                flag_end_rule_as = False
                
                # Para Ignorar los casos de CAST ('' AS CHAR(40)) AS ALIAS_CAMPO
                if (token.value.upper() not in cast_data_type_list and token.value != subquery_alias):
                    flag_rule_select = False
                    column_alias     = token.value
                    rule_select_list = [str_rule_agg, column_alias]
                    rule_select_list_total.append(rule_select_list)
                                        
                    str_rule_agg = ''
                    count_open_parentheses_rule  = 0
                    count_close_parentheses_rule = 0
                    flag_count_parentheses_rule = False
                    last_keyword = 'SELECT'
                

        # Para la ultima logica antes del FROM, si no tiene un AS
        elif (flag_rule_select and token.value == 'FROM' \
              and last_token not in dates_list \
              and flag_rule_filter == False \
              and count_open_parentheses_rule == 0 \
              and token.value != subquery_alias):
            
            flag_rule_select = False            
            column_alias     = last_token
            
            str_rule_agg = str_rule_agg.replace(column_alias, '')
                    
            rule_select_list = [str_rule_agg, column_alias]
            rule_select_list_total.append(rule_select_list)
            str_rule_agg = ''          
            count_open_parentheses_rule  = 0
            count_close_parentheses_rule = 0
            flag_count_parentheses_rule = False    
            
            
        ## Apertura del WHERE, ON para los casos que vienen uno seguido de otro.
        if (((token.is_keyword and token.value.upper() in functions_rules) or \
            (token is Name and token.value.upper() in functions_rules)) and \
            flag_rule_select == False):     
            flag_rule_filter = True  

            if (column_alias == last_token):
                column_alias = None
        
        # Contar los parentesis de las reglas en un SELECT    
        if flag_rule_select:
            if (token.value == '('):
                count_open_parentheses_rule += 1
            elif (token.value == ')'):
                count_close_parentheses_rule += 1
                
            if (count_open_parentheses_rule == count_close_parentheses_rule and \
                count_open_parentheses_rule > 0):
                
                flag_count_parentheses_rule = True
                count_open_parentheses_rule  = 0
                count_close_parentheses_rule = 0

        
 
        ## ACUMULADO DE TEXTO REGLA INTERNA
        if ((flag_rule_select or flag_rule_filter) \
            and flag_end_rule_as == False and token.value != column_alias):
            
            if (token.ttype is Punctuation and token.value == '.'):
                str_rule_agg = str_rule_agg + '.'
                
            elif (last_token in ['.']):
                str_rule_agg = str_rule_agg + token.value 
                
            elif (token.is_keyword and token.value.upper() in functions_rules):
                str_rule_agg = str_rule_agg + ' '+ token.value 
                            
            elif (token.ttype is not Text.Whitespace.Newline):
                str_rule_agg = str_rule_agg + ' '+ token.value
                
                if (token.value == '('):
                    flag_open_parentheses_rule = True 
                elif (token.value == ')'):
                    flag_open_parentheses_rule = False 
                  
        #print("\n\n ACUMULADO DE TEXTO REGLA INTERNA =>"+ str_rule_agg)
        
                
    
    #######################  EXTRACCIÓN DE CAMPOS ########################################
        '''
        Sección que recupera los campos usados tanto en el SELECT como en los WHERE, ON 
        Retorna una lista con el formato: [CAMPO, ALIAS]
        '''
        if (token.ttype is Punctuation and last_token in functions_sql):
            last_keyword = last_token
        

        # Para los casos que los campos sean 'CURRENT_TIME', etc
        if token.is_keyword and token.value.upper() in columns_syntax_keywords \
           and flag_rule_select == False and flag_rule_filter == False:
            column = token.value.upper()
            column_list.append(column)

        elif token.is_keyword and token.value.upper() not in keywords_ignored:
            # mantiene el nombre del último valor de last keyword, ej: SELECT, FROM, WHERE, (ORDER) BY
            last_keyword = token.value.upper()

            
        elif token.ttype is Name and token.value not in functions_sql:
            # analiza los nombres de tokens, los nombres de columna y los valores de condiciones
            
            if (last_keyword in functions_before_column or last_keyword in functions_sql) \
               and (last_token not in ['AS'] or last_token in [',']):               
                                
                if token.value.upper() not in functions_sql and \
                   token.value.upper() not in cast_data_type_list and \
                   token.value.upper() != 'FROM':
                                        
                    if str(last_token) == '.':

                        # we have table.column notation example
                        # append column name to the last entry of columns
                        # as it is a table name in fact
                        table_name = column_list[-1]
                        column_list[-1] = '{}.{}'.format(table_name, token)
                        column = column_list[-1]
                        

                    elif last_token in ['AND'] and last_keyword == 'THEN':
                        column = str(token.value)
                        column_list.append(column)
                        
                    ## alias de campos. Ejemplo: A.Monthly_Target_Cd Monthly_Target_Cd
                    elif ((str(column).upper() == last_token or token.value.upper() == last_token) \
                       and last_keyword == 'SELECT'):
                        column_alias = token.value
                        column_list_internal = [column, column_alias]
                        column_alias = None

                    elif (str(token.value) != subquery_alias and \
                          str(token.value) != column_alias ):
                        
                        column = str(token.value)
                        column_list.append(column)

                        
                  
            ## alias de campos      
            elif (last_keyword in functions_before_column \
                    or last_keyword in functions_sql) \
                    and (last_token in ['AS'] or last_token in [',']) \
                    and token.value.upper() not in functions_sql: 
                column_alias = token.value
                column_list_internal = [column, column_alias]
                column_alias = None

                    
            elif last_keyword in ['INTO'] and token.ttype is Punctuation: #and last_token.ttype is Punctuation:
                # INSERT INTO `foo` (col1, `col2`) VALUES (..)
                column = str(token.value).strip('`')     
                column_list.append(column) 
                
        elif token.ttype is Wildcard:
            # handle * wildcard in SELECT part, but ignore count(*)
            # print(last_keyword, last_token, token.value)
            if last_keyword == 'SELECT' and last_token != '(':
                if str(last_token) == '.':
                    # handle SELECT foo.*
                    table_name = column_list[-1]
                    column_list[-1] = '{}.{}'.format(table_name, str(token))
                    column = column_list[-1]                    
                else:
                    column = str(token.value)  
                    column_list.append(column)
                    
        length_list_column = len(column_list_internal)
        
        
        if (length_list_column >= 1 and (column_list_internal not in column_list_total)):
            column_list_total.append(column_list_internal)
            column = None
            column_alias = None

            
            

    ####################### EXTRACCIÓN DE TABLAS  ####################################        
        '''
        Sección que recupera las tablas de una query 
        Retorna una lista con el formato: [esquema, tabla, alias]
        '''  
        
        subquery_alias    = None
        token_value_clean = " ".join(token.value.upper().split())
        
        if (token.ttype is DML or token.ttype is DDL):
            dml_ddl_name = token.value.upper()  
            from_clausule = False 
            
            if(dml_ddl_name == 'CREATE'):
                flag_create_table = True
            
            if(dml_ddl_name == 'SELECT' and flag_create_table):
                flag_create_table = False
                
        
        if token.ttype is Punctuation and token.value == ',' \
            and from_clausule == True:
            last_keyword = 'FROM'
        
        if token.is_keyword and from_clausule == True and \
           token.value.upper() in stop_from_list:            
            from_clausule = False
            last_keyword = None
        
        
        ## Para los casos de cerrar algunos campos en el SELECT que llaman a FROM
        if (token.is_keyword and token.value.upper() == 'AS' \
           and last_token == ')' and not_get_from):
            last_keyword = 'SELECT'
            not_get_from = False
            
        

        if token.is_keyword and token.value.upper() in table_syntax_keywords:
            # keep the name of the last keyword, the next one can be a table name
            last_keyword = " ".join(token.value.upper().split())
            
            ## Para los casos de algunos campos en el SELECT que llaman a FROM
            if token.value.upper() == 'FROM' and last_token in dates_list:
                not_get_from = True
                
            elif(token.value.upper() == 'FROM' and not_get_from == True):
                not_get_from = False
                
            ## Para los casos de que las tablas estén separadas por ,    
            elif(token.value.upper() == 'FROM' and last_token in ["'"+'0'+"'"]):
                from_clausule = False
                not_get_from  = True

            elif(token.value.upper() == 'FROM' and last_token not in["'"+'0'+"'"]):
                from_clausule = True
                
            elif(token.value.upper() in ['WHERE','GROUP']):
                from_clausule = False
                    
   
        elif str(token) == '(' and last_token not in functions_sql:
            # reset the last_keyword for INSERT `foo` VALUES(id, bar) ...
            #last_keyword = None
            pass
        elif token.is_keyword and str(token) in ['FORCE', 'ORDER']:
            # reset the last_keyword for "SELECT x FORCE INDEX" queries and "SELECT x ORDER BY"
            last_keyword = None
        elif token.is_keyword and str(token) == 'SELECT' and last_keyword in ['INTO', 'TABLE']:
            # reset the last_keyword for "INSERT INTO SELECT" and "INSERT TABLE SELECT" queries
            last_keyword = None
        elif token.ttype is Name or token.is_keyword:
            # print([last_keyword, last_token, token.value])
            # analyze the name tokens, column names and where condition values

            if token.ttype is Name and token.value in functions_ignored \
               and from_clausule == True :
                not_get_from = True
                    
                  

            ## De esta lista quité el 'INTO'
            if last_keyword in ['FROM', 'JOIN', 'INNER JOIN', 'LEFT JOIN', 'RIGHT JOIN', 
                                'LEFT OUTER JOIN','RIGHT OUTER JOIN', 'TABLE'] \
                and last_token not in ['AS', 'CREATE'] \
                and token.value not in ['AS', 'SELECT'] \
                and not_get_from == False \
                and flag_create_table == False \
                and flag_rule_filter == False:

                if last_token == '.':
                    # we have database.table notation example
                    # append table name to the last entry of tables
                    # as it is a database name in fact
                    #print("tables[-1] -------------------> " + str(tables))
                    if (len(tables) >= 1):

                        database_name = tables[-1]
                        table = token.value
                        tables[-1] = '{}.{}'.format(database_name, token)
                        last_keyword = 'TABLE' #None
                        table_list = [database_name, table]
                        
                        if (dml_ddl_name not in ['DROP']):
                            table_list_total.append(table_list)
                    
                elif last_token not in [',', last_keyword] and \
                     last_token not in ['ON', last_keyword] and \
                     len(table_list) > 0:
                    # it's not a list of tables, e.g. SELECT * FROM foo, bar
                    # hence, it can be the case of alias without AS, e.g. SELECT * FROM foo bar
                    alias_table = token.value
                    table_list.append(alias_table)

                else: ## database schema
                    table_name = str(token.value.strip('`'))
                    tables.append(table_name)
            
            elif(last_token == 'AS' and len(table_list) > 0):
                alias_table = token.value
                table_list.append(alias_table)

        length_list_table = len(table_list)

        if (length_list_table >= 2 and (table_list not in table_list_total)):
            table_list_total.append(table_list)
            
            
            
        #Variables que acumulan ultimos valores del token y ayudan a analizar la query
        flag_lastjoin_is_subquery = flag_open_parentheses_subq
    
        if (token.ttype is Text.Whitespace.Newline):
            last_token = last_token.upper() 
        else:
            last_token = token.value.upper()
  
        
    ## CUANDO TERMINA EL TOKEN, PODRÍA QUEDAR LÓGICA POR FUERA DESPUÉS DEL ;
    if (rule_generic_list_total and df_final_columns.empty == False): 
                
        df_final_columns = \
                       updateInDataFrameColumnsRules(df_final_columns, rule_generic_list_total) 
    

    if (df_final_tables.empty == False or df_final_columns.empty == False):
        df_merge_table_column_input = mergeDataframeTablesColumnsQuery(df_final_tables, df_final_columns)
    
    return df_merge_table_column_input

## Funciones: Insertar en Dataframes

In [10501]:
def emptyDataframeExtractQuery(file_type):
    '''
    Función que retorna un objeto dataframe vacío para obtener las tablas, campos y reglas de una query    
    Formato ['TIPO_ARCHIVO','ARCHIVO','LINEAS_DE_CODIGO','NUMERO_PASO','SENTENCIA_DML', 
             'ESQUEMA_OUTPUT','TABLA_OUTPUT','MULTINIVEL','ESQUEMA_INPUT','TABLA_INPUT',
             'ALIAS_SUBQUERY', 'COLUMNA','VARIABLE', 'LOGICA_NEGOCIO', 'ALIAS_CAMPO']
    '''
    columnsDF = []

    if (file_type in ['Bteq','StoredProcedures']):
        columnsDF = ['TIPO_ARCHIVO','ARCHIVO','LINEAS_DE_CODIGO','NUMERO_PASO','SENTENCIA_DML', 
                     'ESQUEMA_OUTPUT','TABLA_OUTPUT','MULTINIVEL','ESQUEMA_INPUT','TABLA_INPUT',
                     'ALIAS_SUBQUERY', 'COLUMNA','VARIABLE', 'LOGICA_NEGOCIO', 'ALIAS_CAMPO']

    elif (file_type == 'Vistas'):  
        columnsDF = ['TIPO_ARCHIVO','ARCHIVO','LINEAS_DE_CODIGO','SENTENCIA_DML','ESQUEMA_OUTPUT','TABLA_OUTPUT',
                     'MULTINIVEL', 'ESQUEMA_INPUT', 'TABLA_INPUT','ALIAS_SUBQUERY', 'COLUMNA',
                     'VARIABLE', 'LOGICA_NEGOCIO', 'ALIAS_CAMPO'] 

    df = pd.DataFrame(columns = columnsDF) 
    return df

In [10502]:
def emptyDataframeTablesQuery(data):
    '''
    Función que retorna un objeto dataframe vacío para obtener las tablas de una query
    Formato ['MULTINIVEL','ESQUEMA_INPUT','TABLA_INPUT','ALIAS_TABLA','ALIAS_SUBQUERY']
    '''
    
    columnsDF = ['MULTINIVEL','ESQUEMA_INPUT','TABLA_INPUT','ALIAS_TABLA', 'ALIAS_SUBQUERY']
    
    if data is not None:
        df = pd.DataFrame(columns = columnsDF) 
    else:
        df = pd.DataFrame(data,columns = columnsDF) 
        
    return df

In [10503]:
def emptyDataframeColumnsQuery(data):
    '''
    Función que retorna un objeto dataframe vacío para obtener los campos y reglas de una query    
    Formato ['MULTINIVEL','CAMPOS','LOGICA_NEGOCIO','ALIAS_CAMPO']
    '''
    
    columnsDF = ['MULTINIVEL','CAMPOS','LOGICA_NEGOCIO','ALIAS_CAMPO']
    
    if data is not None:
        df = pd.DataFrame(columns = columnsDF) 
    else:
        df = pd.DataFrame(data,columns = columnsDF) 
        
    return df

In [10504]:
def emptyDataframeTablesColumnsInputQuery(data):
    '''
    Función que retorna un objeto dataframe vacío para obtener los campos y reglas de una query    
    Formato ['MULTINIVEL','ESQUEMA_INPUT','TABLA_INPUT','ALIAS_SUBQUERY',
             'NOMBRE_CAMPO', 'CAMPO_COMPUESTO','LOGICA_NEGOCIO','ALIAS_CAMPO']
    '''
    
    columnsDF = ['MULTINIVEL','ESQUEMA_INPUT','TABLA_INPUT','ALIAS_SUBQUERY',
                 'NOMBRE_CAMPO', 'CAMPO_COMPUESTO','LOGICA_NEGOCIO','ALIAS_CAMPO']
    
    if data is not None:
        df = pd.DataFrame(columns = columnsDF) 
    else:
        df = pd.DataFrame(data,columns = columnsDF) 
        
    return df

## Funciones de procesamiento de archivos

In [10505]:
def remove_files_csv():
    '''
    Función que elimina del directorio local los archivos antiguos de levantamiento
    '''
    for filename in glob.glob("./LAC_Levantamiento_*.csv"):
        os.remove(filename) 

In [10506]:
def fast_scandir_files(path):
    '''
    Función que escanea los directorios y subdirectorios,
    obteniendo como resultado una lista de archivo con,
    formato: ./Directorio/Subdirectorio/archivo
    '''

    file_list = []
    
    for root, dirs, files in os.walk(path):
        for file in files:
            if(file.endswith(".sql") or file.endswith('.txt')):
                file_list.append(os.path.join(root,file))
                
    return file_list

In [10507]:
def folder_contains_name(folder_dir):
    '''
    Función que retorna que tipo de archivo se está procesando
    '''
    
    if (folder_dir.find('Bteq') != -1 or folder_dir.find('BTEQ') != -1):
        return 'Bteq'
    if (folder_dir.find('Vistas') != -1):
        return 'Vistas'
    if (folder_dir.find('StoredProcedures') != -1):
        return 'StoredProcedures'
    
    return 'Bteq'

In [10508]:
def insert_file_errors(MyFile, file_type, archivo_no_procesado):
    '''
    Función que inserta en un archivo que por alguna razón da error
    '''
    MyFile.write("%s" % 'Archivo tipo *'+ str(file_type) + '*: '+archivo_no_procesado+'\n')

In [10509]:
def insert_dataframe_in_csv(df, file_type):
    '''
    Función que crea un archivo de levantamiento desde un dataframe
    '''
    
    filename = 'LAC_Levantamiento_'+str(file_type)+'.csv'
    #df.to_csv(filename, sep='|', index=None, encoding='utf-8')    
    
    with open(filename, 'a') as f:
        df.to_csv(f, sep='|', index=None, header=f.tell()==0)
    

In [10510]:
def processing_files(folder, filename):
    '''
    Función que abre un archivo y lo procesa linea por linea, eliminando sus comentarios 
    y retornando las querys a procesar en una lista
    '''

    queryVista           = ''    
    
    ## Lista de sintaxis SQL que debería tomar cada linea del archivo,
    ## contiene espacio al final porque hay campos que pueden llamarse "Update_dttm"
    ## y tomarlo como una palabra reservada
    dml_dll_syntax_query = [
        'INSERT', 'CREATE ', 
        'UPDATE ', 
        'DROP ', 'DELETE ','DELETE',
        'RENAME '
    ]
    
    if os.path.exists(filename):
        
        archivo         = open(filename, encoding="utf8",errors='ignore')
        textoArchivo    = archivo.read()
        listaQuery      = []
        sum_lines       = ''
        
        ## BTEQ Y STORED PROCEDURES 
        if (folder in ['Bteq','StoredProcedures']):
            
            bteq_text = re.sub('\-\-.*?\n|\/\*.*?\*\/', ' ', textoArchivo)  
                                    
            bteq_text = remove_comments(bteq_text)
                        
            lines = bteq_text.splitlines();
            
            code_lines = str(len(lines))
            
            for line in lines: 
                # elimina espacios a comienzo de la linea y lo edita a mayuscula
                line = line.strip().upper()                
                                    
                # comienza agregando las querys que interesan
                if line.startswith(tuple(dml_dll_syntax_query)): 
                    sum_lines = line
                    if line.endswith(";"):
                        listaQuery.append(line)
                        sum_lines = ''                        
                        
                # insertar en lista cuando termina la query
                elif (line.endswith(";") and sum_lines != ''): 
                    sum_lines = sum_lines + ' '+ line
                    listaQuery.append(sum_lines)  
                    sum_lines = ''      
                    
                # acumula en una variable hasta conseguir un ;
                elif (sum_lines != ''):                
                    sum_lines = sum_lines + ' '+ line
                           
        ## VIEWS
        elif (folder == 'Vistas'):
            # LIMPIA EL CODIGO SQL PARA QUE NO TENGA NINGUN TIPO DE COMENTARIO
            queryVista = re.sub('\-\-.*?\n|\/\*.*?\*\/|(--[^\r\n]*$)', ' ', textoArchivo)   

            queryVista = remove_comments(queryVista)
                        
            lines = queryVista.splitlines();
            
            code_lines = str(len(lines))
            
            queryVista = queryVista.replace('\\', '')  
            queryVista = queryVista.replace('"', '') 
            
            queryVista = queryVista + ";"
            
            listaQuery.append(queryVista)
   
    return listaQuery, code_lines


## INICIO

In [10511]:
def loop_folders_files(ppal_dir):      
    '''
    Función que itera en los directorios para obtener los archivos que se 
    quieren procesar (bteq, sp, vistas)
    '''
    
    ## Elimina los archivos de levantamiento
    remove_files_csv()
    
    MyFile=open('archivos_no_procesados.txt','w')
    
    folder_dir       = None      # Ruta completa de cada carpeta a leer
    file_dir         = None      # Ruta completa del archivo
    result_folder    = None 
    code_lines       = None
    
    # para pruebas   
    folder_file2='./ArchivosProcesar/StoredProcedures/SPs Capacity Flexible.sql'

    
    ## Obtiene los nombres de los archivo de extracción con sus carpetas y subcarpetas.    
    folder_list = fast_scandir_files(ppal_dir)
    folder_list.sort()
    
    for folder_file in folder_list:
        
        ## Retorna nombre corto del tipo de archivo que se está procesando (Bteq, Vista, StoredProcedure)
        result_folder      = folder_contains_name(folder_file)
        df_by_file_querys  = emptyDataframeExtractQuery(result_folder)
     
        #if (folder_file.find('Bteq') != -1 or folder_file.find('Input_Analitico') != -1): 
        #if (folder_file.find('Vistas') != -1):         
        if (folder_file == folder_file):
            if result_folder is not None:

                print(folder_file)

                listaQuerys, code_lines = processing_files(result_folder, folder_file)

                if(len(listaQuerys) == 0):
                    insert_file_errors(MyFile, result_folder, folder_file)

                process_number = 0

                for queryString in listaQuerys:
                    try:
                        if (queryString):    

                            process_number += 1                    

                            queryString    = preprocess_query(queryString) 

                            #print(str(process_number) + " = " + queryString)

                            #if (process_number == 68):
                                #print(queryString)

                            # Retorna todas las tablas input y output en un dataframe
                            df_internal_merge  = get_output_input_tables(result_folder, folder_file, \
                                                            code_lines, queryString, process_number)

                            if (df_internal_merge.empty == False):
                                df_by_file_querys  = df_by_file_querys.append(df_internal_merge, sort=False)

                
                    except Exception as e: 
                        print("Error with file: "+folder_file)
                        print(e)
                        insert_file_errors(MyFile, result_folder, folder_file)
                        pass  
                    
                # Inserta el conjunto completo a los archivos .csv
                insert_dataframe_in_csv(df_by_file_querys, result_folder)
          
    MyFile.close()  

In [10513]:
if __name__ == "__main__":
    now = datetime.datetime.now()
    print (now)
    
    # Ruta principal donde comienza la extracción
    ppal_dir        = './ArchivosProcesar'       
    
    ## Recorre todas las carpetas y subcarpetas para extraer los archivos 
    loop_folders_files(ppal_dir)

    # Elimina y crea las tablas en Teradata    
    create_and_drop_tables_teradata()
        
    # Inserta el contenido de los archivos a las tablas en Teradata
    insert_files_csv_in_teradata()
    
    now = datetime.datetime.now()
    print (now)

2020-05-08 13:26:59.906354
ELIMINANDO Y CREANDO TABLAS..
INSERTANDO ARCHIVOS A LAS TABLAS..

2020-05-08 13:40:18.215084
